In [2]:
import pandas as pd

In [21]:
match_logs_prev_seasons = pd.read_csv(
            "C:/Users/erknud3/fpl-optimization/model/data/Historic_Data/match_logs_prev_seasons.csv"
        )

match_logs_new_season = pd.read_csv("C:/Users/erknud3/fpl-optimization/model/data/New_Season_Data/match_logs_new_season.csv")

In [22]:
match_logs_new_season = match_logs_new_season[["Date", "fbref", "Player", "Season", "Comp", "Round", "Squad", "weighted_npxGI", "Start", "Min"]] 
match_logs_prev_seasons = match_logs_prev_seasons[["Date", "fbref", "Player", "Season", "Comp", "Round", "Squad", "weighted_npxGI", "Start", "Min"]]

In [65]:
match_logs = pd.concat([match_logs_new_season, match_logs_prev_seasons])

In [66]:
# Step 1: Group by "fbref" and count distinct "Season" values
fbref_season_counts = match_logs.groupby('fbref')['Season'].nunique()

# Step 2: Filter for "fbref" values that have exactly 2 distinct "Season" values
fbrefs_with_two_seasons = fbref_season_counts[fbref_season_counts == 2].index

# Step 3: Filter the original dataframe to keep only rows with these "fbref" values
match_logs = match_logs[match_logs['fbref'].isin(fbrefs_with_two_seasons)]


In [67]:
# Remove duplicate rows based on 'fbref', 'Season', and 'Round'
match_logs = match_logs.drop_duplicates(subset=['fbref', 'Season', 'Round'])


In [69]:
master = pd.read_csv("C:/Users/erknud3/fpl-optimization/model/FPL-ID-MAP/master.csv")

In [105]:
master = master.dropna(subset=['24-25'])

In [107]:
master.head(50)

,code,first_name,second_name,web_name,16-17,17-18,18-19,19-20,20-21,21-22,22-23,23-24,24-25,fbref,understat,transfermarkt
62,15157,James,Milner,Milner,208.0,222.0,254.0,200.0,241.0,221.0,274.0,142.0,134.0,2f90f6b8,489.0,3333.0
77,17601,Scott,Carson,Carson,NaN,NaN,NaN,562.0,608.0,593.0,502.0,347.0,344.0,3e1550ee,8158.0,14555.0
80,17761,James,Tarkowski,Tarkowski,59.0,79.0,72.0,84.0,81.0,98.0,199.0,265.0,236.0,15ea812b,1665.0,173504.0
97,18892,Ashley,Young,Young,260.0,277.0,293.0,224.0,NaN,52.0,538.0,588.0,238.0,be927d03,631.0,14086.0
108,19236,John,Ruddy,Ruddy,NaN,NaN,421.0,412.0,453.0,452.0,NaN,NaN,414.0,da8b19e2,785.0,29712.0
142,21205,Tom,Heaton,Heaton,54.0,74.0,67.0,94.0,28.0,509.0,348.0,384.0,374.0,a6de6361,1651.0,34130.0
178,37096,Łukasz,Fabiański,Fabianski,356.0,350.0,400.0,388.0,431.0,408.0,455.0,533.0,521.0,9328b835,706.0,29692.0
186,37642,Jonny,Evans,Evans,431.0,423.0,222.0,162.0,219.0,201.0,249.0,703.0,371.0,f8fcd2a5,807.0,42412.0
206,39155,Adam,Lallana,Lallana,205.0,228.0,250.0,195.0,54.0,53.0,101.0,138.0,463.0,99813635,486.0,43530.0
228,40349,Asmir,Begovic,Begovic,74.0,29.0,24.0,72.0,NaN,481.0,176.0,NaN,624.0,7b4de647,694.0,33873.0


In [70]:
match_logs = match_logs.merge(master[['fbref', 'transfermarkt']], on='fbref', how='left')

In [71]:
match_logs['transfermarkt'] = match_logs['transfermarkt'].fillna(0).astype(int).astype(str)

In [75]:
import unidecode

def clean_player_name(player_name):
    # Convert to lowercase
    player_name = player_name.lower()
    # Remove accents and special characters
    player_name = unidecode.unidecode(player_name)
    # Replace spaces with hyphens
    player_name = player_name.replace(" ", "-")
    return player_name

In [76]:
# Apply the cleaning function to the 'Player' column
match_logs['cleaned_player_name'] = match_logs['Player'].apply(clean_player_name)

# Create the URL by combining the cleaned player name and transfermarkt ID
match_logs['transfermarkt_url'] = (
    "https://www.transfermarkt.com/" + match_logs['cleaned_player_name'] + "/profil/spieler/" + match_logs['transfermarkt']
)

In [80]:
# Drop duplicates to ensure one URL per player based on the 'Player' and 'transfermarkt' columns
unique_players = match_logs[['Player', 'transfermarkt']].drop_duplicates()

# Generate the URLs for unique players
transfermarkt_url = [
    f"https://www.transfermarkt.com/{clean_player_name(player)}/profil/spieler/{int(transfermarkt_id)}"
    for player, transfermarkt_id in zip(unique_players['Player'], unique_players['transfermarkt'])
]

In [83]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Define headers for the request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0"
}

# Create an empty list to store the results
data = []

# Loop through each player URL to extract the joined date
for index, (player, transfermarkt_id) in enumerate(zip(unique_players['Player'], unique_players['transfermarkt']), start=1):
    # Construct the URL for each player
    url = f"https://www.transfermarkt.com/{clean_player_name(player)}/profil/spieler/{int(transfermarkt_id)}"
    
    try:
        # Send the request
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract the joined date using regex
            joined_date = re.search(r"Joined:.*__content\">(.*?)</span>", str(soup))
            
            # If a match is found, clean the date format
            if joined_date:
                joined_date = joined_date.group(1)
                # Convert to 'YYYY-MM-DD' format
                joined_date = pd.to_datetime(joined_date, format='%b %d, %Y').strftime('%Y-%m-%d')
            else:
                joined_date = None
            
            # Append the data to the list (player name, transfermarkt id, joined date)
            data.append([player, transfermarkt_id, joined_date])
            
            # Print progress for successful extraction
            print(f"[{index}/{len(unique_players)}] Successfully extracted joined date for {player}: {joined_date}")
        
        else:
            print(f"[{index}/{len(unique_players)}] Failed to retrieve data for {player}. Status code: {response.status_code}")
    
    except Exception as e:
        print(f"[{index}/{len(unique_players)}] Error occurred for {player}: {e}")

# Convert the collected data into a dataframe
joined_date_df = pd.DataFrame(data, columns=['Player', 'Transfermarkt ID', 'Joined Date'])

# Display the dataframe
print(joined_date_df)


[1/351] Successfully extracted joined date for Erling Haaland: 2022-07-01
[2/351] Successfully extracted joined date for Darwin Núñez: 2022-07-01
[3/351] Successfully extracted joined date for Mohamed Salah: 2017-07-01
[4/351] Successfully extracted joined date for Kevin De Bruyne: 2015-08-30
[5/351] Successfully extracted joined date for Cole Palmer: 2023-09-01
[6/351] Successfully extracted joined date for Diogo Jota: 2020-09-19
[7/351] Successfully extracted joined date for Ali Al Hamadi: 2024-01-29
[8/351] Successfully extracted joined date for Nicolas Jackson: 2023-07-01
[9/351] Successfully extracted joined date for Gabriel Jesus: 2022-07-04
[10/351] Successfully extracted joined date for Marc Guiu: None
[11/351] Successfully extracted joined date for Luis Díaz: 2022-01-30
[12/351] Successfully extracted joined date for Julio Enciso: 2022-07-01
[13/351] Successfully extracted joined date for Alexander Isak: 2022-08-26
[14/351] Successfully extracted joined date for Son Heung-min:

In [91]:
# Ensure the 'Date' column in match_logs and 'Joined Date' in joined_date_df are in datetime format
match_logs['Date'] = pd.to_datetime(match_logs['Date'], format='%Y-%m-%d')
joined_date_df['Joined Date'] = pd.to_datetime(joined_date_df['Joined Date'], format='%Y-%m-%d')

# Merge the match_logs with joined_date_df to include 'Joined Date' in match_logs
merged_logs = match_logs.merge(joined_date_df, left_on='Player', right_on='Player', how='left')

In [92]:
# Filter rows where the 'Date' in match_logs is after 'Joined Date'
filtered_logs = merged_logs[merged_logs['Date'] > merged_logs['Joined Date']]

In [94]:
filtered_logs = filtered_logs[['Date', 'fbref', 'Player', 'Season', 'Comp', 'Round', 'Squad', 'weighted_npxGI', 'Start', 'Min', 'transfermarkt']]

In [101]:
import requests
from bs4 import BeautifulSoup
import re

# Define headers for the request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0"
}

def extract_team_data(url, comp_id):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table with the team information
    table = soup.find('table', class_='stats_table sortable min_width force_mobilize now_sortable')
    teams = []

    # Extract team data from table rows
    for row in table.tbody.find_all('tr'):
        team_data = row.find('td', {'data-stat': 'team'})
        if team_data:
            a_tag = team_data.find('a')
            if a_tag:
                team_id = a_tag['href'].split('/')[3]
                team_name = a_tag.get_text()
                teams.append({'team_id': team_id, 'team_name': team_name, 'comp_id': comp_id})
    
    return teams

In [103]:
# Define competition URLs and their IDs
competition_urls = {
    'Premier League': [
        "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats",
        "https://fbref.com/en/comps/9/Premier-League-Stats"
    ],
    'Championship': [
        "https://fbref.com/en/comps/10/2023-2024/2023-2024-Championship-Stats",
        "https://fbref.com/en/comps/10/Championship-Stats"
    ],
    'Primeira Liga': [
        "https://fbref.com/en/comps/32/2023-2024/2023-2024-Primeira-Liga-Stats",
        "https://fbref.com/en/comps/32/Primeira-Liga-Stats"
    ],
    'La Liga': [
        "https://fbref.com/en/comps/12/2023-2024/2023-2024-La-Liga-Stats",
        "https://fbref.com/en/comps/12/La-Liga-Stats"
    ],
    'Serie A': [
        "https://fbref.com/en/comps/11/2023-2024/2023-2024-Serie-A-Stats",
        "https://fbref.com/en/comps/11/Serie-A-Stats"
    ]
}

# Extract all team data
all_teams = []
for comp_name, urls in competition_urls.items():
    comp_id = re.search(r'comps/(\d+)', urls[0]).group(1)
    for url in urls:
        all_teams.extend(extract_team_data(url, comp_id))

AttributeError: 'NoneType' object has no attribute 'tbody'

In [ ]:
https://fbref.com/en/squads/b8fd03ef/2023-2024/matchlogs/c9/schedule/Manchester-City-Scores-and-Fixtures-Premier-League
https://fbref.com/en/squads/b2b47a98/2023-2024/matchlogs/c9/schedule/Newcastle-United-Scores-and-Fixtures-Premier-League

In [158]:
soup = BeautifulSoup(requests.get("https://fbref.com/en/squads/b8fd03ef/2023-2024/matchlogs/c9/schedule/Manchester-City-Scores-and-Fixtures-Premier-League").text, 'html.parser')

In [163]:
table = soup.find("table", {"id": "matchlogs_for"})

In [165]:
mancity_matchlogs_23_24 = pd.read_html(StringIO(str(table)))[0]

In [167]:
mancity_matchlogs_23_24 = mancity_matchlogs_23_24[['Date', 'Round', 'Venue', 'Result', 'Opponent']]

In [170]:
mancity_matchlogs_23_24.keys()

Index(['Date', 'Round', 'Venue', 'Result', 'Opponent'], dtype='object')

In [218]:
# Total number of games Manchester City played in 2023-2024
total_games_mancity_23_24 = mancity_matchlogs_23_24.shape[0]

In [219]:
# Games Haaland did not play (Min == 0)
games_not_played = match_logs_haaland[match_logs_haaland['Min'] == 0].shape[0]

# Probability of not playing
prob_not_playing = games_not_played / total_games_mancity_23_24


In [220]:
# Games Haaland started (Start == 'Y')
games_started = match_logs_haaland[match_logs_haaland['Start'] == 'Y'].shape[0]

# Probability of starting
prob_starting = games_started / total_games_mancity_23_24


In [221]:
# Games Haaland came on as a substitute (Min > 0 and Start != 'Y')
games_substitute = match_logs_haaland[(match_logs_haaland['Min'] > 0) & 
                                           (match_logs_haaland['Start'] != 'Y')].shape[0]

# Probability of coming on as a substitute
prob_substitute = games_substitute / total_games_mancity_23_24


In [222]:
probabilities = {
    'prob_not_playing': prob_not_playing,
    'prob_starting': prob_starting,
    'prob_substitute': prob_substitute
}

probabilities

{'prob_not_playing': 0.02631578947368421,
 'prob_starting': 0.8421052631578947,
 'prob_substitute': 0.05263157894736842}

In [223]:
prob_not_playing_new = 0
prob_starting_new = 1
prob_substitute_new = 0


In [224]:
# Define weights
prior_weight = total_games_mancity_23_24  # e.g., 38 games from the previous season
new_evidence_weight = 3  # Number of games this season

# Factor to give more weight to the new evidence
weight_factor = 3  # Example factor to make the new evidence more influential

# Adjusted new evidence weight
adjusted_new_evidence_weight = new_evidence_weight * weight_factor

# Calculate posterior probabilities using weighted average
posterior_prob_not_playing = (
    (prob_not_playing * prior_weight + prob_not_playing_new * adjusted_new_evidence_weight) / 
    (prior_weight + adjusted_new_evidence_weight)
)

posterior_prob_starting = (
    (prob_starting * prior_weight + prob_starting_new * adjusted_new_evidence_weight) / 
    (prior_weight + adjusted_new_evidence_weight)
)

posterior_prob_substitute = (
    (prob_substitute * prior_weight + prob_substitute_new * adjusted_new_evidence_weight) / 
    (prior_weight + adjusted_new_evidence_weight)
)

# Display updated probabilities
posterior_probabilities = {
    'posterior_prob_not_playing': posterior_prob_not_playing,
    'posterior_prob_starting': posterior_prob_starting,
    'posterior_prob_substitute': posterior_prob_substitute
}

posterior_probabilities


{'posterior_prob_not_playing': 0.02127659574468085,
 'posterior_prob_starting': 0.8723404255319149,
 'posterior_prob_substitute': 0.0425531914893617}

In [225]:
match_logs_haaland['Date'] = pd.to_datetime(match_logs_haaland['Date'])

# Separate the data
starts = match_logs_haaland[match_logs_haaland['Start'] == 'Y'].copy()
subs = match_logs_haaland[match_logs_haaland['Start'] == 'N'].copy()


In [226]:
import numpy as np

# Define a function to calculate weights based on recency
def calculate_weights(dates):
    today = pd.Timestamp.now()
    # Example: weights inversely proportional to the number of days from the current date
    weights = np.exp(-(today - dates).dt.days / 30)  # Decay factor: more recent games get higher weight
    return weights

# Calculate weights for starts and subs
starts['Weight'] = calculate_weights(starts['Date'])
subs['Weight'] = calculate_weights(subs['Date'])


In [228]:
# Calculate weighted average minutes per start
weighted_avg_start = (starts['Min'] * starts['Weight']).sum() / starts['Weight'].sum()

# Calculate weighted average minutes per substitute
weighted_avg_substitute = (subs['Min'] * subs['Weight']).sum() / subs['Weight'].sum()

# Display results
weighted_avg_start, weighted_avg_substitute

(89.48217857149234, 20.20703871201762)

In [229]:
# Define expected minutes per scenario
expected_minutes_per_start = weighted_avg_start  # Example value
expected_minutes_per_substitute = weighted_avg_substitute  # Example value

# Calculate expected minutes played
expected_minutes_played = (
    (posterior_prob_starting * expected_minutes_per_start) +
    (posterior_prob_substitute * expected_minutes_per_substitute)
)

# Display the result
expected_minutes_played

78.91879572032386

In [235]:
def adjust_probabilities(probabilities, availability):
    # Initial probabilities for a fully available player
    prob_not_playing = probabilities['posterior_prob_not_playing']
    prob_starting = probabilities['posterior_prob_starting']
    prob_substitute = probabilities['posterior_prob_substitute']
    
    # Adjust probabilities based on availability
    adjusted_prob_not_playing = prob_not_playing + (1 - availability) * (1 - prob_not_playing)
    adjusted_prob_starting = prob_starting * availability
    adjusted_prob_substitute = prob_substitute * availability
    
    # Normalize the probabilities to ensure they sum to 1
    total = adjusted_prob_not_playing + adjusted_prob_starting + adjusted_prob_substitute
    
    if total > 0:
        adjusted_prob_not_playing /= total
        adjusted_prob_starting /= total
        adjusted_prob_substitute /= total
    else:
        # Handle the case where total is zero (if availability is zero)
        adjusted_prob_not_playing = 1.0
        adjusted_prob_starting = 0.0
        adjusted_prob_substitute = 0.0

    return {
        'adjusted_prob_not_playing': adjusted_prob_not_playing,
        'adjusted_prob_starting': adjusted_prob_starting,
        'adjusted_prob_substitute': adjusted_prob_substitute
    }

# Example availability values
availabilities = [1.0, 0.75, 0.5, 0.25, 0.0]

# Original probabilities
probabilities = {
    'posterior_prob_not_playing': 0.021,
    'posterior_prob_starting': 0.872,
    'posterior_prob_substitute': 0.043
}

# Calculate adjusted probabilities for different availability levels
adjusted_probs = {avail: adjust_probabilities(probabilities, avail) for avail in availabilities}

# Display adjusted probabilities
adjusted_probs


{1.0: {'adjusted_prob_not_playing': 0.022435897435897436,
  'adjusted_prob_starting': 0.9316239316239315,
  'adjusted_prob_substitute': 0.04594017094017094},
 0.75: {'adjusted_prob_not_playing': 0.2791491596638655,
  'adjusted_prob_starting': 0.6869747899159664,
  'adjusted_prob_substitute': 0.033876050420168065},
 0.5: {'adjusted_prob_not_playing': 0.5273760330578513,
  'adjusted_prob_starting': 0.4504132231404959,
  'adjusted_prob_substitute': 0.022210743801652895},
 0.25: {'adjusted_prob_not_playing': 0.767530487804878,
  'adjusted_prob_starting': 0.22154471544715448,
  'adjusted_prob_substitute': 0.010924796747967478},
 0.0: {'adjusted_prob_not_playing': 1.0,
  'adjusted_prob_starting': 0.0,
  'adjusted_prob_substitute': 0.0}}

In [237]:
# Calculate expected minutes played for each availability level
expected_minutes_played = {}
for avail, probs in adjusted_probs.items():
    expected_minutes_played[avail] = (
        (probs['adjusted_prob_starting'] * expected_minutes_per_start) +
        (probs['adjusted_prob_substitute'] * expected_minutes_per_substitute)
    )

# Display results
expected_minutes_played

{1.0: 84.29205382367314,
 0.75: 62.15653548762454,
 0.5: 40.75276982384199,
 0.25: 20.045061580019837,
 0.0: 0.0}